In [10]:
import pandas as pd
import numpy as np
import json
import glob
import spacy

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim

In [11]:
df_reports = pd.read_csv('/Users/patrickdunnington/Desktop/DS_Capstone/msds_capstone_2024/personal_notebooks/patrick_nb/mongodump_full_snapshot_4_8/reports.csv')
df_incident = pd.read_csv('/Users/patrickdunnington/Desktop/DS_Capstone/msds_capstone_2024/personal_notebooks/patrick_nb/mongodump_full_snapshot_4_8/incidents.csv')

In [12]:
print(df_incident['Alleged developer of AI system'].value_counts().head())

Alleged developer of AI system
["unknown"]     90
["tesla"]       44
["facebook"]    36
["openai"]      29
["google"]      28
Name: count, dtype: int64


In [13]:
# # Fix the format of the 'Alleged deployer of AI system' column
# df_incident['Alleged deployer of AI system'] = df_incident['Alleged deployer of AI system'].apply(lambda x: ', '.join(eval(x)))

# # Fix the format of the 'Alleged developer of AI system' column
# df_incident['Alleged developer of AI system'] = df_incident['Alleged developer of AI system'].apply(lambda x: ', '.join(eval(x)))

# # Fix the format of the 'Alleged harmed or nearly harmed parties' column
# df_incident['Alleged harmed or nearly harmed parties'] = df_incident['Alleged harmed or nearly harmed parties'].apply(lambda x: ', '.join(eval(x)))

# # Display the DataFrame after fixing the format
# df_incident.head()

In [14]:
isolated_rows = df_incident[df_incident['Alleged developer of AI system'] == '["unknown"]']

In [15]:
isolated_rows

,_id,incident_id,date,reports,Alleged deployer of AI system,Alleged developer of AI system,Alleged harmed or nearly harmed parties,description,title
30,ObjectId(625763df343edc875fe63a1d),31,2017-12-03,"[454,455,456,457,458,459,460,461,462,463,464,4...","[""delhi-metro-rail-corporation""]","[""unknown""]","[""delhi-metro-rail-corporation""]","A driverless metro train in Delhi, India crash...",Driverless Train in Delhi Crashes due to Braki...
34,ObjectId(625763df343edc875fe63a21),35,2014-10-18,"[539,540,541,543,544,545,547,548,549,550,551,5...","[""unknown""]","[""unknown""]","[""ibrahim-diallo""]","An employee was laid off, allegedly by an arti...",Employee Automatically Terminated by Computer ...
68,ObjectId(625763e3343edc875fe63a43),69,2015-07-02,"[1240,1241,1243,1244,1245,1246,1247,1248,1249,...","[""skh-metals""]","[""unknown""]","[""ramji-lal""]",A factory robot at the SKH Metals Factory in M...,Worker killed by robot in welding accident at ...
75,ObjectId(625763e4343edc875fe63a4a),76,2020-10-09,[1339],"[""buenos-aires-city-government""]","[""unknown""]","[""buenos-aires-children""]",Buenos Aires city government uses a facial rec...,Live facial recognition is tracking kids suspe...
79,ObjectId(625763e4343edc875fe63a4e),80,2020-10-24,"[1380,1559]","[""inverness-caledonian-thistle-football-club""]","[""unknown""]","[""livestream-viewers""]",In a Scottish soccer match the AI-enabled ball...,AI mistakes referee’s bald head for football —...
...,...,...,...,...,...,...,...,...,...
604,ObjectId(65725d1e1c8e14919a805a45),616,2023-11-27,"[3448,3452,3453,3454,3455,3456,3457,3458,3459,...","[""the-arena-group"",""sports-illustrated""]","[""unknown""]","[""general-public"",""readers-of-sports-illustrat...","Sports Illustrated, managed by The Arena Group...",Sports Illustrated Is Alleged to Have Used AI ...
605,ObjectId(65844bb96716ef9da676462f),617,2023-11-09,[3495],"[""unnamed-male-student""]","[""unknown""]","[""anonymous-female-high-school-students""]","At a high school in Issaquah, Washington, a ma...",Male student allegedly used AI to generate nud...
616,ObjectId(65ae967aa040369f2c75ab5b),628,2024-01-22,"[3602,3608]","[""unknown""]","[""unknown""]","[""president-joe-biden"",""new-hampshire-voters"",...",A robocall imitating President Joe Biden's voi...,Fake Biden Voice in Robocall Misleads New Hamp...
636,ObjectId(65e362fb67df8539735d3f7f),648,2024-02-07,"[3753,3758]","[""unknown-social-media-accounts""]","[""unknown""]","[""voters-in-pakistan"",""imran-khan"",""pti-(pakis...","A purported deepfake audio clip, falsely attri...",Alleged Deepfake Audio of Imran Khan Calls for...


In [16]:
# import nltk
# nltk.download('punkt')

import pandas as pd
import numpy as np
import json
import glob
import spacy

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim

# Convert date column to datetime format
isolated_rows['date'] = pd.to_datetime(isolated_rows['date'])

# Clean dataset
# for i in range(1, 565):
#     isolated_rows.loc[i, 'reportnumber'] = isolated_rows.loc[i, 'reports'].count(",") + 1

# Clean description column
isolated_rows['clean_description'] = isolated_rows['description'].str.replace('ai', '').str.replace('AI', '')

def lemmatization(incidents, allowed_postags=["NOUN","ADJ","VERB","ADV"]):
    nlp = spacy.load("en_core_web_sm", disable = ["parser", "ner"])
    descript_out = []
    for incident in incidents:
        doc = nlp(incident)
        new_descript = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_descript.append(token.lemma_)
        final = ' '.join(new_descript)
        descript_out.append(final)      
    return (descript_out) 

lemmatized_decript = lemmatization(isolated_rows['clean_description'])

def gen_words(incidents):
    final = []
    for incident in incidents:
        new = gensim.utils.simple_preprocess(incident, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_decript)

id2word = corpora.Dictionary(data_words)

corpus = []

for incident in data_words:
    new = id2word.doc2bow(incident)
    corpus.append(new)

# print(corpus[0][0:20])

word = id2word[[0][:1][0]]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
pyLDAvis.save_html(vis, '/Users/patrickdunnington/Desktop/DS_Capstone/msds_capstone_2024/personal_notebooks/patrick_nb/ldavisual_unknown.html')

/var/folders/4b/9y5tt5gx6jsc140h_cssjx1m0000gn/T/ipykernel_2063/1356619043.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isolated_rows['date'] = pd.to_datetime(isolated_rows['date'])
/var/folders/4b/9y5tt5gx6jsc140h_cssjx1m0000gn/T/ipykernel_2063/1356619043.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isolated_rows['clean_description'] = isolated_rows['description'].str.replace('ai', '').str.replace('AI', '')
/Users/patrickdunnington/anaconda3/envs/bayesml/lib/python3.11/site-packages/sklearn

In [17]:
# Tokenize and remove stop words
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem(token) for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

isolated_rows['clean_description'] = isolated_rows['clean_description'].apply(preprocess_text)

# Create document-term matrix
vectorizer = CountVectorizer()
incident_DTM = vectorizer.fit_transform(isolated_rows['clean_description'])

# Perform LDA
lda = LatentDirichletAllocation(n_components=10, random_state=321)
incident_topics = lda.fit_transform(incident_DTM)

# Print top terms for each topic
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_terms = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(f"Topic {topic_idx + 1}: {', '.join(top_terms)}")

Topic 1: author, content, employe, notic, websit, twitter, ceo, manag, compani, remov
Topic 2: face, use, report, voic, alleg, high, resid, plate, suspect, system
Topic 3: video, use, minor, imag, report, alleg, voic, creat, tatum, post
Topic 4: use, facial, recognit, system, arrest, social, firm, media, post, depart
Topic 5: imag, technolog, alleg, recognit, man, use, arrest, potenti, facial, theft
Topic 6: alleg, fake, face, clip, polit, deepfak, audio, circul, thousand, custom
Topic 7: softwar, elect, fals, black, deepfak, report, audio, social, media, breach
Topic 8: use, student, deepfak, voic, school, data, new, video, facial, like
Topic 9: robot, factori, lal, machin, metal, reach, soccer, prompt, make, handl
Topic 10: deepfak, alleg, video, use, post, polic, technolog, due, imag, ukrnian


/var/folders/4b/9y5tt5gx6jsc140h_cssjx1m0000gn/T/ipykernel_2063/3673670255.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isolated_rows['clean_description'] = isolated_rows['clean_description'].apply(preprocess_text)
